In [1]:
!python libraries.py

2024-08-28 15:06:54,389 - INFO - Upgrading pip...
2024-08-28 15:06:54,389 - INFO - Running pip command: install --upgrade pip
2024-08-28 15:06:56,116 - INFO - Command 'install --upgrade pip' executed successfully.
2024-08-28 15:06:56,117 - INFO - pip upgraded successfully.
2024-08-28 15:06:56,117 - INFO - Installing library numpy...
2024-08-28 15:06:56,117 - INFO - Running pip command: install numpy
2024-08-28 15:06:57,440 - INFO - Command 'install numpy' executed successfully.
2024-08-28 15:06:57,440 - INFO - numpy installed successfully.
2024-08-28 15:06:57,440 - INFO - Installing library pandas...
2024-08-28 15:06:57,440 - INFO - Running pip command: install pandas
2024-08-28 15:06:58,614 - INFO - Command 'install pandas' executed successfully.
2024-08-28 15:06:58,614 - INFO - pandas installed successfully.
2024-08-28 15:06:58,614 - INFO - Installing library matplotlib...
2024-08-28 15:06:58,614 - INFO - Running pip command: install matplotlib
2024-08-28 15:06:59,713 - INFO - Comman

In [2]:
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6244120110601901947
xla_global_id: -1
]


In [4]:
import os
import pandas as pd
import numpy as np
import utils

In [5]:
data_dir = './data'
measurement_data_dir = f'{data_dir}/measurement'
os.listdir(measurement_data_dir)

trips_dir = os.path.join(measurement_data_dir, 'trips')
os.listdir(trips_dir)

csvs_list = [f for f in os.listdir(trips_dir) if f.endswith('.csv')]
trips = utils.csvs2dfs(
    trips_dir=trips_dir, 
    csvs_list=csvs_list
)

for i in range(len(trips)):
    trips[i].columns = [col.replace('_', '') for col in trips[i].columns]

bounds_SOC = utils.data_cleaning(
    dfs=trips,
    steady_cols_to_drop=['minSoC', 'maxSoC']
)

min_SOC = bounds_SOC[0]
max_SOC = bounds_SOC[1]

In [6]:
# Target value
y = []
for i in range(len(trips)):
    yd = trips[i]['SoC'].values
    # drop della colonna poiche' ho gia' salvato i valori in y_train cosi' da rendere piu' semplice la costruzione di X_train
    trips[i].drop(['SoC'], axis=1, inplace=True)
    y.append(yd)

y_shapes = []
for i in range(len(y)):
    y_shapes.append(y[i].shape)
y_shapes

[(10090,),
 (14130,),
 (6706,),
 (4124,),
 (13667,),
 (31645,),
 (20933,),
 (28059,),
 (18344,),
 (14176,),
 (14245,),
 (16384,),
 (7160,),
 (6946,),
 (22348,),
 (19066,),
 (6687,),
 (8808,),
 (15876,),
 (17196,),
 (19829,),
 (18388,),
 (10426,),
 (5322,),
 (7627,),
 (16122,),
 (20005,),
 (17436,),
 (13390,),
 (13974,),
 (11870,),
 (16722,),
 (32518,),
 (16113,),
 (15794,),
 (29550,),
 (10195,),
 (13521,),
 (22899,),
 (29140,),
 (56102,),
 (20233,),
 (7534,),
 (32256,),
 (3545,),
 (38220,),
 (18223,),
 (15286,),
 (15610,),
 (11095,),
 (11911,),
 (14029,),
 (10397,),
 (11993,),
 (11133,),
 (9780,),
 (10219,),
 (8050,),
 (14690,),
 (13665,),
 (9686,),
 (9209,),
 (10969,),
 (7958,),
 (5480,),
 (7338,),
 (13626,),
 (28523,),
 (14173,),
 (16429,)]

In [7]:
# Feature data
X = []
for i in range(len(trips)):
    xd = trips[i].values
    X.append(xd)

X_shapes = []
for i in range(len(X)):
    X_shapes.append(X[i].shape)
X_shapes

[(10090, 19),
 (14130, 19),
 (6706, 19),
 (4124, 19),
 (13667, 19),
 (31645, 19),
 (20933, 19),
 (28059, 19),
 (18344, 19),
 (14176, 19),
 (14245, 19),
 (16384, 19),
 (7160, 19),
 (6946, 19),
 (22348, 19),
 (19066, 19),
 (6687, 19),
 (8808, 19),
 (15876, 19),
 (17196, 19),
 (19829, 19),
 (18388, 19),
 (10426, 19),
 (5322, 19),
 (7627, 19),
 (16122, 19),
 (20005, 19),
 (17436, 19),
 (13390, 19),
 (13974, 19),
 (11870, 19),
 (16722, 19),
 (32518, 19),
 (16113, 19),
 (15794, 19),
 (29550, 19),
 (10195, 19),
 (13521, 19),
 (22899, 19),
 (29140, 19),
 (56102, 19),
 (20233, 19),
 (7534, 19),
 (32256, 19),
 (3545, 19),
 (38220, 19),
 (18223, 19),
 (15286, 19),
 (15610, 19),
 (11095, 19),
 (11911, 19),
 (14029, 19),
 (10397, 19),
 (11993, 19),
 (11133, 19),
 (9780, 19),
 (10219, 19),
 (8050, 19),
 (14690, 19),
 (13665, 19),
 (9686, 19),
 (9209, 19),
 (10969, 19),
 (7958, 19),
 (5480, 19),
 (7338, 19),
 (13626, 19),
 (28523, 19),
 (14173, 19),
 (16429, 19)]

In [8]:
# Normalizzazione delle feature
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
normalized_X = []
for i in range(len(X)):
    nx = scaler.fit_transform(X[i])
    normalized_X.append(nx)

normalized_X_shapes = []
for i in range(len(normalized_X)):
    normalized_X_shapes.append(normalized_X[i].shape)
normalized_X_shapes

[(10090, 19),
 (14130, 19),
 (6706, 19),
 (4124, 19),
 (13667, 19),
 (31645, 19),
 (20933, 19),
 (28059, 19),
 (18344, 19),
 (14176, 19),
 (14245, 19),
 (16384, 19),
 (7160, 19),
 (6946, 19),
 (22348, 19),
 (19066, 19),
 (6687, 19),
 (8808, 19),
 (15876, 19),
 (17196, 19),
 (19829, 19),
 (18388, 19),
 (10426, 19),
 (5322, 19),
 (7627, 19),
 (16122, 19),
 (20005, 19),
 (17436, 19),
 (13390, 19),
 (13974, 19),
 (11870, 19),
 (16722, 19),
 (32518, 19),
 (16113, 19),
 (15794, 19),
 (29550, 19),
 (10195, 19),
 (13521, 19),
 (22899, 19),
 (29140, 19),
 (56102, 19),
 (20233, 19),
 (7534, 19),
 (32256, 19),
 (3545, 19),
 (38220, 19),
 (18223, 19),
 (15286, 19),
 (15610, 19),
 (11095, 19),
 (11911, 19),
 (14029, 19),
 (10397, 19),
 (11993, 19),
 (11133, 19),
 (9780, 19),
 (10219, 19),
 (8050, 19),
 (14690, 19),
 (13665, 19),
 (9686, 19),
 (9209, 19),
 (10969, 19),
 (7958, 19),
 (5480, 19),
 (7338, 19),
 (13626, 19),
 (28523, 19),
 (14173, 19),
 (16429, 19)]

In [9]:
# array di indici random per la scelta dei trip da utilizzare per la validation
val_dataset_perc = .2
val_dataset_dim = int(val_dataset_perc * len(normalized_X))
val_dataset_idx = np.random.randint(low=0, high=len(normalized_X), size=val_dataset_dim)
print(f'len(val_dataset_idx) = {len(val_dataset_idx)}')
val_dataset_idx.sort()
val_dataset_idx

len(val_dataset_idx) = 14


array([15, 16, 16, 16, 17, 23, 36, 38, 40, 43, 49, 50, 54, 62])

In [10]:
# array di indici random per la scelta dei trip rimanenti
all_dataset_idx = np.arange(start=0, stop=len(normalized_X), step=1)
all_dataset_idx = np.array(list(set(all_dataset_idx) - set(val_dataset_idx)))

In [11]:
# array di indici random per la scelta dei trip da utilizzare per il testing
test_dataset_perc = .05
test_dataset_dim = int(test_dataset_perc * len(all_dataset_idx))
test_dataset_idx = np.random.randint(low=0, high=len(all_dataset_idx), size=test_dataset_dim)
print(f'len(test_dataset_idx) = {len(test_dataset_idx)}')
test_dataset_idx.sort()
test_dataset_idx

len(test_dataset_idx) = 2


array([ 1, 51])

In [12]:
# array di indici random per la scelta dei trip da utilizzare per il training
train_dataset_idx = np.array(list(set(all_dataset_idx) - set(test_dataset_idx)))
print(f'len(train_dataset_idx) = {len(train_dataset_idx)}')
train_dataset_idx.sort()
train_dataset_idx

len(train_dataset_idx) = 56


array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 18, 19, 20,
       21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 39, 41,
       42, 44, 45, 46, 47, 48, 52, 53, 55, 56, 57, 58, 59, 60, 61, 63, 64,
       65, 66, 67, 68, 69])

In [13]:
data_idx_path = './data_idx'

# Salvataggio degli indici da utilizzare successivamente per il training
train_dataset_idx_filename = 'train_dataset_idx.txt'
with open(f'./{data_idx_path}/{train_dataset_idx_filename}', 'w') as file:
    for i in range(len(train_dataset_idx)):
        file.write(f"{train_dataset_idx[i]}\n")

# Salvataggio degli indici da utilizzare successivamente per la validation
val_dataset_idx_filename = 'val_dataset_idx.txt'
with open(f'./{data_idx_path}/{val_dataset_idx_filename}', 'w') as file:
    for i in range(len(val_dataset_idx)):
        file.write(f"{val_dataset_idx[i]}\n")

# Salvataggio degli indici da utilizzare successivamente per il testing
test_dataset_idx_filename = 'test_dataset_idx.txt'
with open(f'./{data_idx_path}/{test_dataset_idx_filename}', 'w') as file:
    for i in range(len(test_dataset_idx)):
        file.write(f"{test_dataset_idx[i]}\n")

In [14]:
# FNN parameters
no_features = normalized_X[0].shape[1]
no_responses = 1
no_hidden_units = 55

epochs = 10
no_training = 3
batch_size = 64

In [15]:
from tensorflow import keras
from tensorflow.keras import layers
from custom import CustomLeakyReLU, CustomClippedReLU

In [16]:
model = keras.Sequential([
    layers.Input(shape=(no_features,)),
    layers.Dense(
        no_hidden_units,
        activation=keras.activations.tanh
    ),
    layers.Dense(
        no_hidden_units,
        activation=CustomLeakyReLU(negative_slope=0.3)
    ),
    layers.Dense(
        no_responses,
        activation=CustomClippedReLU()
    )
])

In [17]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=20000,
    decay_rate=0.9,
    staircase=False
)

optimizer = keras.optimizers.SGD(
    learning_rate=lr_schedule
)

In [18]:
model.compile(
    optimizer=optimizer,
    loss='mse'
)

In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 55)                  │           1,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 55)                  │           3,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              56 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,236 (16.55 KB)

 Trainable params: 4,236 (16.55 KB)

 Non-trainable params: 0 (0.00 B)